In [ ]:
### GETS INFO ABOUT FIRST SEED MATCH
import requests
import pandas as pd

# Function to fetch sequences in batches
def fetch_sequences_batch(ensembl_ids):
    url = "https://rest.ensembl.org/sequence/id"
    headers = {"Content-Type": "application/json"}
    data = {"ids": ensembl_ids}
    response = requests.post(url, headers=headers, json=data)
    if response.status_code == 200:
        return response.json()
    else:
        return None

# Function to convert only 'T' to 'U' in the DNA sequence
def dna_to_rna(dna_sequence):
    return dna_sequence.replace('T', 'U')

pd.set_option('display.max_colwidth', None)

# Batch size and list of Ensembl IDs from the filtered DataFrame
batch_size = 50
ensembl_ids = list(df_filtered.index)
region_length = 50

for i in range(0, len(ensembl_ids), batch_size):
    batch_ids = ensembl_ids[i:i + batch_size]
    sequences = fetch_sequences_batch(batch_ids)
    
    if sequences:
        for seq_info in sequences:
            sequence = seq_info['seq']
            ensembl_id = seq_info['id']
            # Check if 'AATCCTAT' is in the sequence
            if 'AATCCTAT' in sequence:
                match_index = sequence.find('AATCCTAT')
                # Extract region around the match
                start = max(0, match_index - region_length)
                end = min(len(sequence), match_index + len('AATCCTAT') + region_length)
                
                region_dna = sequence[start:end]
                # Convert only 'T' to 'U' in the region
                region_rna = dna_to_rna(region_dna)
                sequence_rna = dna_to_rna(sequence)
                
                # Store the RNA sequence and other details in the DataFrame
                df_filtered.loc[ensembl_id, 'region_around_match'] = region_rna
                df_filtered.loc[ensembl_id, 'sequence_length'] = len(sequence)
                
                # Print the region converted to RNA
                print(f"Ensembl ID: {ensembl_id}")
                print(f"Sequence Length: {len(sequence)}")
                print(f"Region around match (DNA): {region_dna}")
                print(f"Region around match (RNA): {region_rna}")
                print(f"{sequence_rna}")
                
                # Stop after one result
                break
    if 'region_around_match' in df_filtered.columns:
        break


In [ ]:
## MAKING STRUCTURAL PREDICTIONS WITH CHAI
from pathlib import Path
import numpy as np
import torch
from chai_lab.chai1 import run_inference
import os
gpu_index = "0"
os.environ["CUDA_VISIBLE_DEVICES"] = gpu_index
os.environ["OMP_NUM_THREADS"] = "6"


example_fasta = f"""
>protein|PIK3CB_AUAGGAUUCAUAUUAGGAGAU_AUCUCCUAAUAUGAAUCCUAU 
MYSGAGPALAPPAPPPPIQGYAFKPPPRPDFGTSGRTIKLQANFFEMDIPKIDIYHYELDIKPEKCPRRVNREIVEHMVQHFKTQIFGDRKPVFDGRKNLYTAMPLPIGRDKVELEVTLPGEGKDRIFKVSIKWVSCVSLQALHDALSGRLPSVPFETIQALDVVMRHLPSMRYTPVGRSFFTASEGCSNPLGGGREVWFGFHQSVRPSLWKMMLNIDVSATAFYKAQPVIEFVCEVLDFKSIEEQQKPLTDSQRVKFTKEIKGLKVEITHCGQMKRKYRVCNVTRRPASHQTFPLQQESGQTVECTVAQYFKDRHKLVLRYPHLPCLQVGQEQKHTYLPLEVCNIVAGQRCIKKLTDNQTSTMIRATARSAPDRQEEISKLMRSADFNTDPYVREFGIMVKDEMTDVTGRVLQPPSILYGGRNKAIATPVQGVWDMRNKQFHTGIEIKVWAIACFAPQRQCTEVHLKSFTEQLRKISRDAGMPIQGQPCFCKYAQGADSVEPMFRHLKNTYAGLQLVVVILPGKTPVYAEVKRVGDTVLGMATQCVQMKNVQRTTPQTLSNLCLKINVKLGGVNNILLPQGRPPVFQQPVIFLGADVTHPPAGDGKKPSIAAVVGSMDAHPNRYCATVRVQQHRQEIIQDLAAMVRELLIQFYKSTRFKPTRIIFYRAGVSEGQFQQVLHHELLAIREACIKLEKDYQPGITFIVVQKRHHTRLFCTDKNERVGKSGNIPAGTTVDTKITHPTEFDFYLCSHAGIQGTSRPSHYHVLWDDNRFSSDELQILTYQLCHTYVRCTRSVSIPAPAYYAHLVAFRARYHLVDKEHDAAEGDHTDGQANGRDHQALAKAVQVHQDTLRTMYFA
>rna|PIK3CB_AUAGGAUUCAUAUUAGGAGAU_AUCUCCUAAUAUGAAUCCUAU 
AUAGGAUUCAUAUUAGGAGAU
>rna|PIK3CB_AUAGGAUUCAUAUUAGGAGAU_AUCUCCUAAUAUGAAUCCUAU 
AUCUCCUAAUAUGAAUCCUAU 
""".strip()

### ACCUCUGUUGUAGGACUAAUCCUAU
### ...
# Write the example FASTA to a file ### REQUIRED
fasta_path = Path("example.fasta")
fasta_path.write_text(example_fasta)


output_dir = Path("outputs")
output_cif_paths = run_inference(
    fasta_file=fasta_path,
    output_dir=output_dir,
    num_trunk_recycles=3,
    num_diffn_timesteps=200,
    seed=42,
    device=torch.device("cuda:0"),  # Use "cpu" if no GPU is available
    use_esm_embeddings=True,
)


pdb_files = list(output_dir.glob("*.pdb"))
print("Generated PDB files:")
for pdb_file in pdb_files:
    print(pdb_file)


In [9]:
### COLLECTING MATCHED SEQUENCES
import pandas as pd
import requests

# Load the data
file_path = '6048D_rawCounts.txt'
df = pd.read_csv(file_path, sep='\t', index_col=0)

# Columns of interest
unmod_cols = ['MR10_unmod_1', 'MR11_unmod_2', 'MR12_unmod_3']
nt_cols = ['MR1_NT_1', 'MR2_NT_2', 'MR3_NT_3']
amide_cols = ['MR4_Amide3_1', 'MR5_Amide3_2', 'MR6_Amide3_3']
gna_cols = ['MR7_GNA7_1', 'MR8_GNA7_2', 'MR9_GNA7_3']

# Function to fetch sequences from Ensembl
def fetch_sequences_batch(ensembl_ids):
    url = "https://rest.ensembl.org/sequence/id"
    headers = {"Content-Type": "application/json"}
    data = {"ids": ensembl_ids}
    response = requests.post(url, headers=headers, json=data)
    if response.status_code == 200:
        return response.json()
    else:
        return None

# Prepare to search for sequences with "AATCCTAT"
batch_size = 50
ensembl_ids = list(df.index)
matches = []

# Iterate through the Ensembl IDs in batches
for i in range(0, len(ensembl_ids), batch_size):
    batch_ids = ensembl_ids[i:i + batch_size]
    sequences = fetch_sequences_batch(batch_ids)
    
    if sequences:
        for seq_info in sequences:
            sequence = seq_info['seq']
            ensembl_id = seq_info['id']
            if 'AATCCTAT' in sequence:
                matches.append(ensembl_id)

# Output matched Ensembl IDs to file
with open("matched_sequences_AATCCTAT.txt", "w") as file:
    for match in matches:
        file.write(f"{match}\n")

# Print the number of matches
print(f"Total matches found: {len(matches)}")


Total matches found: 11254


In [11]:
import pandas as pd
import numpy as np

# Step 1: Read ENS IDs to exclude from the matched_sequences_AATCCTAT.txt file
excluded_ens_ids = []
with open('matched_sequences_AATCCTAT.txt', 'r') as f:
    excluded_ens_ids = [line.strip() for line in f.readlines()]

# Step 2: Read the raw counts data from 6048D_rawCounts.txt
df = pd.read_csv('6048D_rawCounts.txt', sep='\t', index_col=0)

# Step 3: Filter out ENS IDs from matched_sequences_AATCCTAT.txt
initial_count = df.shape[0]
df_filtered = df[~df.index.isin(excluded_ens_ids)]
final_count = df_filtered.shape[0]

# Print number of entries before and after removal
print(f"Initial number of entries: {initial_count}")
print(f"Number of entries after removal of matched ENS IDs: {final_count}")

# Step 4: Select columns of interest (NT and Amide columns only)
columns_of_interest = [
    'MR1_NT_1', 'MR2_NT_2', 'MR3_NT_3',
    'MR4_Amide3_1', 'MR5_Amide3_2', 'MR6_Amide3_3'
]
df_counts = df_filtered[columns_of_interest]

# Step 5: Normalize the counts
total_counts = df_counts.sum(axis=0)
scaling_factors = total_counts / total_counts.mean()
df_normalized = df_counts.div(scaling_factors, axis=1)

# Step 6: Calculate mean counts for each group
nt_cols = ['MR1_NT_1', 'MR2_NT_2', 'MR3_NT_3']
amide_cols = ['MR4_Amide3_1', 'MR5_Amide3_2', 'MR6_Amide3_3']

df_normalized['mean_nt'] = df_normalized[nt_cols].mean(axis=1)
df_normalized['mean_amide'] = df_normalized[amide_cols].mean(axis=1)

# Step 7: Calculate Log2 Fold Changes (Amide vs NT)
df_normalized['log2FC_amide_vs_nt'] = np.log2(df_normalized['mean_amide'] / df_normalized['mean_nt'])

# Step 8: Calculate counts where log2FC > 0.25 or log2FC < -0.25 for Amide vs NT
amide_log2fc = df_normalized['log2FC_amide_vs_nt']
amide_positive = (amide_log2fc > 0.25).sum()
amide_negative = (amide_log2fc < -0.25).sum()
amide_total = amide_positive + amide_negative

# Step 9: Print the results for Amide vs NT before the mask
print("\nAmide vs NT (before mask):")
print(f"Number of data points with log2FC > 0.25: {amide_positive}")
print(f"Number of data points with log2FC < -0.25: {amide_negative}")
print(f"Total number of data points with |log2FC| > 0.25: {amide_total}")

# Step 10: Apply the mask to filter data where mean NT counts are greater than 10^1.5
mask = df_normalized['mean_nt'] > 10**1.5
df_masked = df_normalized[mask]

# Step 11: Recalculate counts where log2FC > 0.25 or log2FC < -0.25 for Amide vs NT after the mask
amide_log2fc_masked = df_masked['log2FC_amide_vs_nt']
amide_positive_masked = (amide_log2fc_masked > 0.25).sum()
amide_negative_masked = (amide_log2fc_masked < -0.25).sum()
amide_total_masked = amide_positive_masked + amide_negative_masked

# Step 12: Print the results for Amide vs NT after applying the mask
print("\nAmide vs NT (after mask):")
print(f"Number of data points with log2FC > 0.25: {amide_positive_masked}")
print(f"Number of data points with log2FC < -0.25: {amide_negative_masked}")
print(f"Total number of data points with |log2FC| > 0.25: {amide_total_masked}")


Initial number of entries: 58884
Number of entries after removal of matched ENS IDs: 47630

Amide vs NT (before mask):
Number of data points with log2FC > 0.25: 13261
Number of data points with log2FC < -0.25: 15170
Total number of data points with |log2FC| > 0.25: 28431

Amide vs NT (after mask):
Number of data points with log2FC > 0.25: 877
Number of data points with log2FC < -0.25: 722
Total number of data points with |log2FC| > 0.25: 1599


/data/home/mrichte3/miniconda3/lib/python3.12/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log2
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [ ]:
### DECENT ALIGNMENT, BUT HAS THESE SHORT SEQUENCES. CANNOT FIGURE OUT THE ISSUE
import pandas as pd
import requests
from Bio import pairwise2
from Bio.pairwise2 import format_alignment
from multiprocessing import Pool, cpu_count
import time

def fetch_sequences_batch(ensembl_ids):
    url = "https://rest.ensembl.org/sequence/id"
    headers = {"Content-Type": "application/json"}
    data = {"ids": ensembl_ids}
    response = requests.post(url, headers=headers, json=data)
    if response.status_code == 200:
        return response.json()
    else:
        return None

def scan_for_high_similarity(seq_info):
    sequence = seq_info['seq']
    ensembl_id = seq_info['id']
    seed_sequence = "AATCCTAT"
    
    # Start timing
    start_time = time.time()

    # Check if seed sequence is present (side check)
    contains_seed = seed_sequence in sequence
    
    # Perform alignment
    alignments = pairwise2.align.localms(sequence, target_sequence, 2, -1, -5, -2)
    best_alignment = alignments[0] if alignments else None
    
    # Calculate elapsed time
    elapsed_time = time.time() - start_time
    
    if best_alignment:
        score = best_alignment[2]
        start = best_alignment[3]
        end = best_alignment[4]
        alignment_str = format_alignment(*best_alignment)
        return f"{ensembl_id}: Sequence length={len(sequence)}, Contains seed={contains_seed}, Alignment score={score:.2f}, Alignment range=({start}, {end}), Time elapsed={elapsed_time:.2f}s\n{alignment_str}"
    else:
        return f"{ensembl_id}: Sequence length={len(sequence)}, Contains seed={contains_seed}, No significant alignment found, Time elapsed={elapsed_time:.2f}s"

target_sequence = "ATCTCCTAATATGAATCCTAT"

df = pd.read_csv('6048D_rawCounts.txt', sep='\t', index_col=0)

ensembl_ids = list(df.index)
ensembl_ids = ["ENSG00000100811", "ENSG00000100811"]
batch_size = 50
batch_ids = ensembl_ids[:batch_size]

sequences = fetch_sequences_batch(batch_ids)

if sequences:
    num_cores = cpu_count()
    print(f"Using {num_cores} CPU cores for parallel processing")
    with Pool(num_cores) as pool:
        for result in pool.imap_unordered(scan_for_high_similarity, sequences):
            print(result)


In [5]:
## GOOD SETTINGS FOR NCBI BLAST-LIKE ALIGNMENT
import pandas as pd
import requests
from Bio import Align
import time

def fetch_single_sequence(ensembl_id):
    url = "https://rest.ensembl.org/sequence/id"
    headers = {"Content-Type": "application/json"}
    data = {"ids": [ensembl_id]}  # Requesting only the single sequence
    response = requests.post(url, headers=headers, json=data)
    if response.status_code == 200:
        return response.json()[0]  # Return the single sequence from response
    else:
        return None

def scan_for_high_similarity(seq_info):
    sequence = seq_info['seq']
    ensembl_id = seq_info['id']
    seed_sequence = "AATCCTAT"
    
    # Start timing
    start_time = time.time()

    # Check if seed sequence is present (side check)
    contains_seed = seed_sequence in sequence
    
    # Perform alignment using Bio.Align.PairwiseAligner
    aligner = Align.PairwiseAligner()
    aligner.mode = 'local'  # Local alignment like BLAST
    aligner.match_score = 2  # Match score of +2
    aligner.mismatch_score = -3  # Mismatch score of -3 (BLAST-like)
    aligner.open_gap_score = -5  # Gap opening penalty like BLAST
    aligner.extend_gap_score = -2  # Gap extension penalty like BLAST

    print("started alignment")
    alignments = aligner.align(sequence, target_sequence)
    print("ended alignment")
    
    best_alignment = alignments[0] if alignments else None
    
    # Calculate elapsed time
    elapsed_time = time.time() - start_time
    
    if best_alignment:
        aligned_seq = best_alignment.target  # The aligned sequence from the gene
        aligned_target = best_alignment.query  # The aligned sequence from the target
        score = best_alignment.score
        aligned_positions = best_alignment.aligned

        print(f"Alignment details:\n")
        display_alignment(sequence, target_sequence, aligned_positions)
        return f"{ensembl_id}: Alignment score={score:.2f}, Time elapsed={elapsed_time:.2f}s"
    else:
        return f"{ensembl_id}: Sequence length={len(sequence)}, Contains seed={contains_seed}, No significant alignment found, Time elapsed={elapsed_time:.2f}s"

def display_alignment(sequence, target_sequence, alignment_positions):
    target_alignments = alignment_positions[1]  # Target sequence positions
    gene_alignments = alignment_positions[0]  # Gene sequence positions

    for i, (target_range, gene_range) in enumerate(zip(target_alignments, gene_alignments)):
        target_aligned = target_sequence[target_range[0]:target_range[1]]
        gene_aligned = sequence[gene_range[0]:gene_range[1]]
        print(f"{target_aligned} {gene_aligned}")
        break

target_sequence = "ATCTCCTAATATGAATCCTAT"

# Fetch a single sequence
ensembl_id = "ENSG00000044459"

sequence_data = fetch_single_sequence(ensembl_id)
print("data rcvd")

if sequence_data:
    # Process the single sequence
    result = scan_for_high_similarity(sequence_data)
    print(result)


data rcvd
started alignment
ended alignment
Alignment details:

TCTCCTAATA TCTCCTAATA
ENSG00000044459: Alignment score=23.00, Time elapsed=0.28s


In [ ]:
## WILL PRINT THE FIRST ALIGNMENT ENTRY, NO TRIMMING
import pandas as pd
import requests
from Bio import pairwise2
from Bio.pairwise2 import format_alignment
import time

def fetch_single_sequence(ensembl_id):
    url = "https://rest.ensembl.org/sequence/id"
    headers = {"Content-Type": "application/json"}
    data = {"ids": [ensembl_id]}
    response = requests.post(url, headers=headers, json=data)
    if response.status_code == 200:
        return response.json()[0]
    else:
        return None

def scan_for_high_similarity(seq_info):
    sequence = seq_info['seq']
    ensembl_id = seq_info['id']
    seed_sequence = "AATCCTAT"
    start_time = time.time()
    contains_seed = seed_sequence in sequence
    alignments = pairwise2.align.globalmx(sequence, target_sequence, 2, -1)
    elapsed_time = time.time() - start_time

    if alignments:
        best_alignment = alignments[0]
        aligned_seq, aligned_target, score, begin, end = best_alignment
        alignment_str = format_alignment(aligned_seq, aligned_target, score, begin, end, full_sequences=False)

        # Trim each line to a maximum of 50 characters
        trimmed_alignment = "\n".join([line[:50] for line in alignment_str.splitlines()])

        # Print all available data
        print(f"Alignment Data for {ensembl_id}:")
        print(f"Alignment Start: {begin}")
        print(f"Alignment End: {end}")
        print(f"Alignment Score: {score}")
        print(f"Trimmed Alignment:\n{trimmed_alignment}")

        # Now iterate over all alignments and print them in trimmed form
        print("\nAll Alignments (trimmed to 50 characters per line):")
        for a in alignments:
            alignment_str = format_alignment(*a)
            trimmed_a = "\n".join([line for line in alignment_str.splitlines()])
            print(trimmed_a)
            break
        
        return f"{ensembl_id}: Sequence length={len(sequence)}, Contains seed={contains_seed}, Alignment score={score:.2f}, Alignment range=({begin}, {end}), Time elapsed={elapsed_time:.2f}s"
    else:
        return f"{ensembl_id}: Sequence length={len(sequence)}, Contains seed={contains_seed}, No significant alignment found, Time elapsed={elapsed_time:.2f}s"

target_sequence = "ATCTCCTAATATGAATCCTAT"
ensembl_id = "ENSG00000051382"
sequence_data = fetch_single_sequence(ensembl_id)
print("Data received")

if sequence_data:
    result = scan_for_high_similarity(sequence_data)
    print(result)


In [ ]:
## WILL PRINT ALL THE RESULTS, BUT TRIMS EACH ONE
import pandas as pd
import requests
from Bio import pairwise2
from Bio.pairwise2 import format_alignment
import time

def fetch_single_sequence(ensembl_id):
    url = "https://rest.ensembl.org/sequence/id"
    headers = {"Content-Type": "application/json"}
    data = {"ids": [ensembl_id]}
    response = requests.post(url, headers=headers, json=data)
    if response.status_code == 200:
        return response.json()[0]
    else:
        return None

def scan_for_high_similarity(seq_info):
    sequence = seq_info['seq']
    ensembl_id = seq_info['id']
    seed_sequence = "AATCCTAT"
    start_time = time.time()
    contains_seed = seed_sequence in sequence
    alignments = pairwise2.align.globalmx(sequence, target_sequence, 2, -1)
    elapsed_time = time.time() - start_time

    if alignments:
        best_alignment = alignments[0]
        aligned_seq, aligned_target, score, begin, end = best_alignment
        alignment_str = format_alignment(aligned_seq, aligned_target, score, begin, end, full_sequences=False)

        # Trim each line to a maximum of 50 characters
        trimmed_alignment = "\n".join([line[:50] for line in alignment_str.splitlines()])

        # Print all available data
        print(f"Alignment Data for {ensembl_id}:")
        print(f"Alignment Start: {begin}")
        print(f"Alignment End: {end}")
        print(f"Alignment Score: {score}")
        print(f"Trimmed Alignment:\n{trimmed_alignment}")

        # Now iterate over all alignments and print them in trimmed form
        print("\nAll Alignments (trimmed to 50 characters per line):")
        for a in alignments:
            alignment_str = format_alignment(*a)
            trimmed_a = "\n".join([line[:50] for line in alignment_str.splitlines()])
            print(trimmed_a)
        
        return f"{ensembl_id}: Sequence length={len(sequence)}, Contains seed={contains_seed}, Alignment score={score:.2f}, Alignment range=({begin}, {end}), Time elapsed={elapsed_time:.2f}s"
    else:
        return f"{ensembl_id}: Sequence length={len(sequence)}, Contains seed={contains_seed}, No significant alignment found, Time elapsed={elapsed_time:.2f}s"

target_sequence = "ATCTCCTAATATGAATCCTAT"
ensembl_id = "ENSG00000051382"
sequence_data = fetch_single_sequence(ensembl_id)
print("Data received")

if sequence_data:
    result = scan_for_high_similarity(sequence_data)
    print(result)


In [ ]:
import requests
from Bio.Blast import NCBIWWW, NCBIXML

def get_entrez_id_from_ensembl(ensembl_id):
    url = f"https://rest.ensembl.org/xrefs/id/{ensembl_id}?external_db=EntrezGene"
    headers = {"Content-Type": "application/json"}
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        data = response.json()
        if data:
            return data[0]["primary_id"]  # Return the Entrez Gene ID
    return None

ensembl_id = "ENSG00000100811" ###   "ENSG00000051382" ###  
entrez_id = get_entrez_id_from_ensembl(ensembl_id)

if entrez_id:
    print(f"Fetched Entrez ID for {ensembl_id}: {entrez_id}")
    target_sequence = "ATCTCCTAATATGAATCCTAT"

    print(f"Running BLAST using Entrez ID {entrez_id} as the subject...")

    # Run BLAST using the Entrez ID
    result_handle = NCBIWWW.qblast("blastn", "nt", target_sequence, entrez_query=f"{entrez_id} AND txid9606[ORGN]", expect=10, short_query=True)

    with open("blast_result.xml", "w") as out_handle:
        out_handle.write(result_handle.read())

    with open("blast_result.xml") as result_handle:
        blast_record = NCBIXML.read(result_handle)

    # Only get the first alignment and its score
    if blast_record.alignments:
        alignment = blast_record.alignments[0]
        print(f"Alignment: {alignment.title}")
        hsp = alignment.hsps[0]  # First HSP (High Scoring Pair)
        print(f"Score: {hsp.score}")
        print(f"Query/Match/Subject:\n{hsp.query}\n{hsp.match}\n{hsp.sbjct}")
        # print("\n" + "-"*60 + "\n")
else:
    print(f"Could not fetch the Entrez ID for Ensembl ID {ensembl_id}")


In [13]:
import requests
from Bio.Blast import NCBIWWW, NCBIXML

def get_entrez_id_from_ensembl(ensembl_id):
    url = f"https://rest.ensembl.org/xrefs/id/{ensembl_id}?external_db=EntrezGene"
    headers = {"Content-Type": "application/json"}
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        data = response.json()
        if data:
            return data[0]["primary_id"]
    return None

ensembl_id = "ENSG00000100811"
entrez_id = get_entrez_id_from_ensembl(ensembl_id)

if entrez_id:
    print(f"Fetched Entrez ID for {ensembl_id}: {entrez_id}")
    target_sequence = "ATCTCCTAATATGAATCCTAT"

    print(f"Running BLAST using Entrez ID {entrez_id} as the subject...")

    result_handle = NCBIWWW.qblast("blastn", "nt", target_sequence, entrez_query=f"{entrez_id} AND txid9606[ORGN]", expect=10, short_query=True)
    print("result rcvd from blast")

    with open("blast_result.xml", "w") as out_handle:
        out_handle.write(result_handle.read())

    with open("blast_result.xml") as result_handle:
        blast_record = NCBIXML.read(result_handle)

    if blast_record.alignments:
        alignment = blast_record.alignments[0]
        print(f"Alignment: {alignment.title}")
        hsp = alignment.hsps[0]
        print(f"Score: {hsp.score}")

        aligned_query = hsp.query
        aligned_gene = hsp.sbjct
        match_positions = hsp.match
        start = hsp.sbjct_start
        end = hsp.sbjct_end

        gene_sequence = alignment.hsps[0].sbjct
        target_len = len(target_sequence)
        aligned_len = len(aligned_query)

        before_len = start - 1
        after_len = target_len - aligned_len - before_len

        unmatched_before = gene_sequence[:before_len] if before_len > 0 else ""
        unmatched_after = gene_sequence[end:end + after_len] if after_len > 0 else ""

        print(f"Query/Match/Subject:\n{aligned_query}\n{match_positions}\n{aligned_gene}")
        print(f"Unmatched Gene Sequence Before Alignment: {unmatched_before}")
        print(f"Unmatched Gene Sequence After Alignment: {unmatched_after}")
else:
    print(f"Could not fetch the Entrez ID for Ensembl ID {ensembl_id}")


Fetched Entrez ID for ENSG00000100811: 7528
Running BLAST using Entrez ID 7528 as the subject...
result rcvd from blast
Alignment: gi|1653960621|ref|NM_003403.5| Homo sapiens YY1 transcription factor (YY1), mRNA
Score: 10.0
Query/Match/Subject:
ATGAATCCTA
||||||||||
ATGAATCCTA
Unmatched Gene Sequence Before Alignment: ATGAATCCTA
Unmatched Gene Sequence After Alignment: 


In [23]:
from Bio import Entrez
target_sequence = "ATCTCCTAATATGAATCCTAT"
Entrez.email = "1800420@gmail.com"
ensembl_id = "ENSG00000100811"

if blast_record.alignments:
    alignment = blast_record.alignments[0]
    
    accession = alignment.accession
    print(f"Accession number: {accession}")
    
    handle = Entrez.efetch(db="nucleotide", id=accession, rettype="fasta", retmode="text")
    full_gene_sequence = handle.read().split("\n", 1)[1].replace("\n", "")
    
    gene_length = len(full_gene_sequence)
    print(f"Full gene sequence length: {gene_length}")
    
    aligned_region = alignment.hsps[0].sbjct
    print(f"Aligned region: {aligned_region}")
    
    match_position = full_gene_sequence.find(aligned_region)
    
    if match_position != -1:
        print(f"Aligned region found at position: {match_position + 1}")
        print(f"Aligned region ends at position: {match_position + len(aligned_region)}")
        
        target_length = len(target_sequence)
        aligned_length = len(aligned_region)

        bases_needed_before = target_length - aligned_length - 1  # 10 before
        bases_needed_after = 1  # 1 after
        
        start_position = max(0, match_position - bases_needed_before)
        end_position = min(gene_length, match_position + aligned_length + bases_needed_after)

        complete_region = full_gene_sequence[start_position:end_position]

        print(f"Complete region of gene corresponding to target sequence: {complete_region}")
    else:
        print("Aligned region not found in the full gene sequence.")
else:
    print("No alignment found in the BLAST result.")


Accession number: NM_003403
Full gene sequence length: 6534
Aligned region: ATGAATCCTA
Aligned region found at position: 1454
Aligned region ends at position: 1463
Complete region of gene corresponding to target sequence: AATTTTAAAAATGAATCCTAC


In [2]:
## GOOD SETTINGS FOR NCBI BLAST-LIKE ALIGNMENT
import pandas as pd
import requests
from Bio import Align
import time

def fetch_single_sequence(ensembl_id):
    url = "https://rest.ensembl.org/sequence/id"
    headers = {"Content-Type": "application/json"}
    data = {"ids": [ensembl_id]}  # Requesting only the single sequence
    response = requests.post(url, headers=headers, json=data)
    if response.status_code == 200:
        return response.json()[0]  # Return the single sequence from response
    else:
        return None

def scan_for_high_similarity(seq_info):
    sequence = seq_info['seq']
    ensembl_id = seq_info['id']
    seed_sequence = "AATCCTAT"
    
    # Start timing
    start_time = time.time()

    # Check if seed sequence is present (side check)
    contains_seed = seed_sequence in sequence
    
    # Perform alignment using Bio.Align.PairwiseAligner
    aligner = Align.PairwiseAligner()
    aligner.mode = 'local'  # Local alignment like BLAST
    aligner.match_score = 2  # Match score of +2
    aligner.mismatch_score = -3  # Mismatch score of -3 (BLAST-like)
    aligner.open_gap_score = -5  # Gap opening penalty like BLAST
    aligner.extend_gap_score = -2  # Gap extension penalty like BLAST

    print("started alignment")
    alignments = aligner.align(sequence, target_sequence)
    print("ended alignment")
    
    best_alignment = alignments[0] if alignments else None
    
    # Calculate elapsed time
    elapsed_time = time.time() - start_time
    
    if best_alignment:
        aligned_seq = best_alignment.target  # The aligned sequence from the gene
        aligned_target = best_alignment.query  # The aligned sequence from the target
        score = best_alignment.score
        aligned_positions = best_alignment.aligned

        print(f"Alignment details:\n")
        display_alignment(sequence, target_sequence, aligned_positions)
        return f"{ensembl_id}: Alignment score={score:.2f}, Time elapsed={elapsed_time:.2f}s"
    else:
        return f"{ensembl_id}: Sequence length={len(sequence)}, Contains seed={contains_seed}, No significant alignment found, Time elapsed={elapsed_time:.2f}s"

def display_alignment(sequence, target_sequence, alignment_positions):
    target_alignments = alignment_positions[1]  # Target sequence positions
    gene_alignments = alignment_positions[0]  # Gene sequence positions

    for i, (target_range, gene_range) in enumerate(zip(target_alignments, gene_alignments)):
        target_aligned = target_sequence[target_range[0]:target_range[1]]
        gene_aligned = sequence[gene_range[0]:gene_range[1]]
        print(f"{target_aligned} {gene_aligned}")

target_sequence = "ATCTCCTAATATGAATCCTAT"

# Fetch a single sequence
ensembl_id = "ENSG00000100811"
sequence_data = fetch_single_sequence(ensembl_id)
print("data rcvd")

if sequence_data:
    # Process the single sequence
    result = scan_for_high_similarity(sequence_data)
    print(result)


data rcvd
started alignment
ended alignment
Alignment details:

AATATGAATCCTA AAAATGAATCCTA
ENSG00000100811: Alignment score=21.00, Time elapsed=0.02s


In [ ]:
# ALIGNS SEQUENCES OFFLINE, MAPS IDS BACK TO THE ORIGINAL. prints results first, then id
import csv
from Bio import Align
import sys
csv.field_size_limit(sys.maxsize)
# Load the mapping between original_id and updated_id
def load_id_mapping(mapping_file):
    id_map = {}
    with open(mapping_file, 'r') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            id_map[row['updated_id']] = row['original_id']
    return id_map

def scan_for_high_similarity(ensembl_id, sequence):
    seed_sequence = "AATCCTAT"
    
    aligner = Align.PairwiseAligner()
    aligner.mode = 'local'
    aligner.match_score = 2
    aligner.mismatch_score = -3
    aligner.open_gap_score = -5
    aligner.extend_gap_score = -2

    alignments = aligner.align(sequence, target_sequence)
    
    best_alignment = alignments[0] if alignments else None
    
    if best_alignment:
        aligned_seq = best_alignment.target
        aligned_target = best_alignment.query
        score = best_alignment.score
        aligned_positions = best_alignment.aligned

        display_alignment(sequence, target_sequence, aligned_positions)
        return f"{ensembl_id}: Alignment score={score:.2f}"
    else:
        return f"{ensembl_id}: No significant alignment found"

def display_alignment(sequence, target_sequence, alignment_positions):
    target_alignments = alignment_positions[1]  # Target sequence positions
    gene_alignments = alignment_positions[0]  # Gene sequence positions

    for i, (target_range, gene_range) in enumerate(zip(target_alignments, gene_alignments)):
        target_aligned = target_sequence[target_range[0]:target_range[1]]
        gene_aligned = sequence[gene_range[0]:gene_range[1]]
        print(f"{target_aligned} {gene_aligned}")
        # break
######################code2
    gene_aligned = ""
    for gene_range in gene_alignments:
        gene_aligned += sequence[gene_range[0]:gene_range[1]]
    print(f"{gene_aligned}")

# Load the original to updated ID map
id_mapping = load_id_mapping('updated_ids.csv')

input_files = ['sequences_from_updated.csv', 'sequences.csv']
target_sequence = "ATCTCCTAATATGAATCCTAT" # 271
                    
             

for input_file in input_files:
    with open(input_file, 'r') as csvfile:
        reader = csv.DictReader(csvfile)
        
        for row in reader:
            ensembl_id = row['ensemble_id']
            sequence = row['sequence']
            
            # Use original_id for sequences_from_updated.csv
            if input_file == 'sequences_from_updated.csv':
                ensembl_id = id_mapping.get(ensembl_id, ensembl_id)

            result = scan_for_high_similarity(ensembl_id, sequence)
            print(result)

            # break  # Break after the first entry


In [ ]:
### DOES ALIGNMENTS, BUT OUTPUTS THE WRONG IDS FOR UPDATED DATA. USES UPDATED IDS. ALSO DOESN'T USE BEST ALIGNMENTS. NO IDEA WHAT HAPPENED
import csv
from Bio import Align
import sys
csv.field_size_limit(sys.maxsize)
def load_id_mapping(mapping_file):
    id_map = {}
    with open(mapping_file, 'r') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            id_map[row['updated_id']] = row['original_id']
    return id_map

def scan_for_high_similarity(ensembl_id, sequence, csvwriter):
    aligner = Align.PairwiseAligner()
    aligner.mode = 'local'
    aligner.match_score = 2
    aligner.mismatch_score = -3
    aligner.open_gap_score = -5
    aligner.extend_gap_score = -2

    alignments = aligner.align(sequence, target_sequence)
    best_alignment = alignments[0] if alignments else None
    
    if best_alignment:
        aligned_seq = best_alignment.target
        aligned_target = best_alignment.query
        aligned_positions = best_alignment.aligned
        gene_aligned = display_alignment(sequence, target_sequence, aligned_positions)
        print(f"{ensembl_id} {gene_aligned}")
        # csvwriter.writerow([ensembl_id, gene_aligned])

def display_alignment(sequence, target_sequence, alignment_positions):
    gene_aligned = ""
    gene_alignments = alignment_positions[0]
    for gene_range in gene_alignments:
        gene_aligned += sequence[gene_range[0]:gene_range[1]]
    return gene_aligned

id_mapping = load_id_mapping('updated_ids.csv')
input_files = ['sequences.csv', 'sequences_from_updated.csv']
# input_files = ['sequences_from_updated.csv', 'sequences.csv']
target_sequence = "ATCTCCTAATATGAATCCTAT"


with open('gene_alignments.csv', 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(['ensembl_id', 'gene_aligned'])  # Write header

    for input_file in input_files:
        with open(input_file, 'r') as infile:
            reader = csv.DictReader(infile)
            
            for row in reader:
                ensembl_id = row['ensemble_id']
                sequence = row['sequence']
                
                if input_file == 'sequences_from_updated.csv':
                    ensembl_id = id_mapping.get(ensembl_id, ensembl_id)

                scan_for_high_similarity(ensembl_id, sequence, csvwriter)
                # break  # Break after the first entry


In [2]:
import pandas as pd
df = pd.read_csv('gene_alignments.csv')
df['target_rna'] = df['gene_aligned'].apply(lambda seq: seq.replace('T', 'U'))
df.to_csv('gene_alignments2.csv', index=False)


In [ ]:
# ALIGNMENTS NCBI all results
import requests
from Bio.Blast import NCBIWWW, NCBIXML

def get_entrez_id_from_ensembl(ensembl_id):
    url = f"https://rest.ensembl.org/xrefs/id/{ensembl_id}?external_db=EntrezGene"
    headers = {"Content-Type": "application/json"}
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        data = response.json()
        if data:
            return data[0]["primary_id"]  # Return the Entrez Gene ID
    return None

ensembl_id = "ENSG00000051382"
entrez_id = get_entrez_id_from_ensembl(ensembl_id)

if entrez_id:
    print(f"Fetched Entrez ID for {ensembl_id}: {entrez_id}")
    target_sequence = "ATCTCCTAATATGAATCCTAT"

    print(f"Running BLAST using Entrez ID {entrez_id} as the subject...")

    # Add taxonomic filter for Homo sapiens (taxid:9606)
    # Set expect threshold and other relevant parameters for a short sequence
    result_handle = NCBIWWW.qblast("blastn", "nt", target_sequence, entrez_query=f"{entrez_id} AND txid9606[ORGN]", expect=10, short_query=True)

    with open("blast_result.xml", "w") as out_handle:
        out_handle.write(result_handle.read())

    with open("blast_result.xml") as result_handle:
        blast_record = NCBIXML.read(result_handle)

    for alignment in blast_record.alignments:
        print(f"Alignment: {alignment.title}")
        for hsp in alignment.hsps:
            print(f"Query/Match/Subject:\n{hsp.query}\n{hsp.match}\n{hsp.sbjct}")
            print("\n" + "-"*60 + "\n")
else:
    print(f"Could not fetch the Entrez ID for Ensembl ID {ensembl_id}")


In [ ]:
# ALIGNMENTS NCBI
import requests
from Bio.Blast import NCBIWWW, NCBIXML

def get_entrez_id_from_ensembl(ensembl_id):
    url = f"https://rest.ensembl.org/xrefs/id/{ensembl_id}?external_db=EntrezGene"
    headers = {"Content-Type": "application/json"}
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        data = response.json()
        if data:
            return data[0]["primary_id"]  # Return the Entrez Gene ID
    return None

ensembl_id = "ENSG00000051382"
entrez_id = get_entrez_id_from_ensembl(ensembl_id)

if entrez_id:
    print(f"Fetched Entrez ID for {ensembl_id}: {entrez_id}")
    target_sequence = "ATCTCCTAATATGAATCCTAT"

    print(f"Running BLAST using Entrez ID {entrez_id} as the subject...")

    result_handle = NCBIWWW.qblast("blastn", "nt", target_sequence, entrez_query=f"{entrez_id} AND txid9606[ORGN]", expect=1, short_query=True)

    with open("blast_result.xml", "w") as out_handle:
        out_handle.write(result_handle.read())

    with open("blast_result.xml") as result_handle:
        blast_record = NCBIXML.read(result_handle)

    # Initialize variables to store the highest scoring alignment and HSP
    best_hsp = None
    best_alignment = None
    highest_score = 0
    print("ncbi data rcvd")
    # Iterate through all alignments and HSPs
    for alignment in blast_record.alignments:
        for hsp in alignment.hsps:
            if hsp.score > highest_score:
                highest_score = hsp.score
                best_hsp = hsp
                best_alignment = alignment

    # Print the alignment with the highest score
    if best_hsp and best_alignment:
        print(f"Best Alignment: {best_alignment.title}")
        print(f"Highest Score: {best_hsp.score}")
        print(f"Query/Match/Subject:\n{best_hsp.query}\n{best_hsp.match}\n{best_hsp.sbjct}")
        print("\n" + "-"*60 + "\n")
else:
    print(f"Could not fetch the Entrez ID for Ensembl ID {ensembl_id}")


In [ ]:
# ALIGNMENTS NCBI
import requests
from Bio.Blast import NCBIWWW, NCBIXML

def get_entrez_id_from_ensembl(ensembl_id):
    url = f"https://rest.ensembl.org/xrefs/id/{ensembl_id}?external_db=EntrezGene"
    headers = {"Content-Type": "application/json"}
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        data = response.json()
        if data:
            return data[0]["primary_id"]  # Return the Entrez Gene ID
    return None

ensembl_id = "ENSG00000051382"
entrez_id = get_entrez_id_from_ensembl(ensembl_id)

if entrez_id:
    print(f"Fetched Entrez ID for {ensembl_id}: {entrez_id}")
    target_sequence = "ATCTCCTAATATGAATCCTAT"

    print(f"Running BLAST using Entrez ID {entrez_id} and limiting to Homo sapiens...")

    # Run BLAST using the Entrez ID and limiting to Homo sapiens (taxid:9606)
    result_handle = NCBIWWW.qblast(
        "blastn", 
        "nt", 
        target_sequence, 
        entrez_query=f"txid9606[ORGN]",  # Specify the organism (Homo sapiens, taxid:9606)
        entrez_query_filter=entrez_id  # Use only Entrez ID 5291
    )

    with open("blast_result.xml", "w") as out_handle:
        out_handle.write(result_handle.read())

    with open("blast_result.xml") as result_handle:
        blast_record = NCBIXML.read(result_handle)

    # Initialize variables to store the highest scoring alignment and HSP
    best_hsp = None
    best_alignment = None
    highest_score = 0

    # Iterate through all alignments and HSPs
    for alignment in blast_record.alignments:
        for hsp in alignment.hsps:
            if hsp.score > highest_score:
                highest_score = hsp.score
                best_hsp = hsp
                best_alignment = alignment

    # Print the alignment with the highest score
    if best_hsp and best_alignment:
        print(f"Best Alignment: {best_alignment.title}")
        print(f"Highest Score: {best_hsp.score}")
        print(f"Query/Match/Subject:\n{best_hsp.query}\n{best_hsp.match}\n{best_hsp.sbjct}")
        print("\n" + "-"*60 + "\n")
else:
    print(f"Could not fetch the Entrez ID for Ensembl ID {ensembl_id}")


In [1]:
# PREFETCH SEQUENCES FROM UPDATED IDS
import requests
import csv

# Define the input and output file names
input_ids_file = 'updated_ids.csv'  # This is the updated IDs file
output_file = 'sequences_from_updated.csv'  # New output file for sequences

# Step 1: Read the updated Ensembl IDs from updated_ids.csv
updated_ensembl_ids = []

with open(input_ids_file, 'r') as f:
    reader = csv.DictReader(f)
    for row in reader:
        updated_id = row['updated_id']
        updated_ensembl_ids.append(updated_id)

# Step 2: Divide updated Ensembl IDs into batches of 50
batch_size = 50
batches = [updated_ensembl_ids[i:i + batch_size] for i in range(0, len(updated_ensembl_ids), batch_size)]

# Step 3: Fetch sequences and write to a new CSV
with open(output_file, 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(['ensembl_id', 'sequence'])  # Write header

    for batch in batches:
        # Prepare the request to the Ensembl REST API
        server = "https://rest.ensembl.org"
        endpoint = "/sequence/id"
        headers = {"Content-Type": "application/json", "Accept": "application/json"}
        data = {"ids": batch}

        # Make the POST request
        response = requests.post(server + endpoint, headers=headers, json=data)

        if not response.ok:
            print(f"Error fetching sequences for batch: {batch}")
            print(f"Response: {response.text}")
            response.raise_for_status()

        # Parse the JSON response
        sequences = response.json()

        # Write each sequence to the CSV file
        for item in sequences:
            ensembl_id = item['id']
            sequence = item['seq']
            csvwriter.writerow([ensembl_id, sequence])

print(f"Sequences have been written to {output_file}")


Sequences have been written to sequences_from_updated.csv


In [9]:
# TESTING: COLLECTS MISSING SEQUENCES USING SELENIUM
import os
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
import time
import re  # Import regex for extracting the new ID

os.environ['MOZ_HEADLESS'] = '1'

options = Options()
options.headless = True
service = Service('/data/home/mrichte3/.local/bin/geckodriver')

driver = webdriver.Firefox(service=service, options=options)

# Navigate to the Ensembl website
driver.get("https://useast.ensembl.org/index.html")

time.sleep(2)  # Allow the page to load

# Find the search input box using XPath and enter the Ensembl ID
search_box = driver.find_element(By.XPATH, "/html/body/div[1]/div/div[2]/div[1]/div/div[1]/div[1]/div[1]/div[2]/div/form/fieldset/div[3]/div[1]/div/input[2]")
ensembl_id = "ENSG00000112096"
search_box.send_keys(ensembl_id)

# Find and click the search button using XPath
search_button = driver.find_element(By.XPATH, "/html/body/div[1]/div/div[2]/div[1]/div/div[1]/div[1]/div[1]/div[2]/div/form/fieldset/div[3]/div[1]/input")
search_button.click()

time.sleep(5)  # Wait for the search results page to load

# Look for the new identifier in the specified XPath
try:
    result = driver.find_element(By.XPATH, "/html/body/div[1]/div/div[2]/div[1]/div/div[2]/div[2]/div[4]/div/div/div[2]/div/div/div[3]/div[2]/div[1]/div[2]/div/div[1]/div/div/div/div[3]").text
    
    # Use regex to extract all Ensembl IDs (first and second)
    ids = re.findall(r'ENSG\d+', result)
    
    if len(ids) > 1:
        new_id = ids[1]  # Capture the second occurrence
        print(f"New Ensembl ID: {new_id}")
    else:
        print(f"Could not find the second Ensembl ID in the result.")

except Exception as e:
    print(f"Error or no new identifier found: {e}")

driver.quit()


New Ensembl ID: ENSG00000291237


In [ ]:
# COLLECTS MISSING SEQUENCES, BUT ERRORS OUT
import csv
import requests
import os
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
import time
import re
import pandas as pd
import sys
csv.field_size_limit(sys.maxsize)
input_file = '6048D_rawCounts.txt'
sequences_file = 'sequences.csv'
output_file = 'sequences_fixed.csv'

ensembl_ids = []
with open(input_file, 'r') as f:
    header = f.readline()
    for line in f:
        if line.strip():
            parts = line.strip().split()
            if parts:
                ensembl_id = parts[0]
                ensembl_ids.append(ensembl_id)

existing_sequences = {}
with open(sequences_file, 'r') as f:
    reader = csv.reader(f)
    next(reader)
    for row in reader:
        if len(row) == 2:
            ensembl_id, sequence = row
            existing_sequences[ensembl_id] = sequence

missing_ids = [ensembl_id for ensembl_id in ensembl_ids if ensembl_id not in existing_sequences]

os.environ['MOZ_HEADLESS'] = '1'
options = Options()
options.headless = True
service = Service('/data/home/mrichte3/.local/bin/geckodriver')
driver = webdriver.Firefox(service=service, options=options)

def fetch_updated_ensembl_id(ensembl_id):
    driver.get("https://useast.ensembl.org/index.html")
    time.sleep(2)
    search_box = driver.find_element(By.XPATH, "/html/body/div[1]/div/div[2]/div[1]/div/div[1]/div[1]/div[1]/div[2]/div/form/fieldset/div[3]/div[1]/div/input[2]")
    search_box.send_keys(ensembl_id)
    search_button = driver.find_element(By.XPATH, "/html/body/div[1]/div/div[2]/div[1]/div/div[1]/div[1]/div[1]/div[2]/div/form/fieldset/div[3]/div[1]/input")
    search_button.click()
    time.sleep(5)
    try:
        result = driver.find_element(By.XPATH, "/html/body/div[1]/div/div[2]/div[1]/div/div[2]/div[2]/div[4]/div/div/div[2]/div/div/div[3]/div[2]/div[1]/div[2]/div/div[1]/div/div/div/div[3]").text
        ids = re.findall(r'ENSG\d+', result)
        if len(ids) > 1:
            new_id = ids[1]
            print(f"Updated ID found: {new_id} for {ensembl_id}")
            return new_id
    except Exception as e:
        print(f"Error fetching ID for {ensembl_id}: {e}")
    return None

updated_ids = []
for missing_id in missing_ids:
    updated_id = fetch_updated_ensembl_id(missing_id)
    if updated_id:
        updated_ids.append({'original_id': missing_id, 'updated_id': updated_id})

driver.quit()

df = pd.DataFrame(updated_ids)
# df.to_csv('updated_ids.csv', index=False)


In [18]:
# COLLECTS MISSING SEQUENCES 2, STARTING FROM ABOVE DFS
import os
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
import time
import re
import pandas as pd

# Assuming these lists are already available in memory
# updated_ids = [{'original_id': '...', 'updated_id': '...'}, ...]
# missing_ids = ['...', '...']
last_processed_id = 'ENSG00000281167'
skip = True

os.environ['MOZ_HEADLESS'] = '1'
options = Options()
options.headless = True
service = Service('/data/home/mrichte3/.local/bin/geckodriver')
driver = webdriver.Firefox(service=service, options=options)

def fetch_updated_ensembl_id(ensembl_id):
    try:
        driver.get("https://useast.ensembl.org/index.html")
        time.sleep(2)
        search_box = driver.find_element(By.XPATH, "/html/body/div[1]/div/div[2]/div[1]/div/div[1]/div[1]/div[1]/div[2]/div/form/fieldset/div[3]/div[1]/div/input[2]")
        search_box.send_keys(ensembl_id)
        search_button = driver.find_element(By.XPATH, "/html/body/div[1]/div/div[2]/div[1]/div/div[1]/div[1]/div[1]/div[2]/div/form/fieldset/div[3]/div[1]/input")
        search_button.click()
        time.sleep(5)
        
        result = driver.find_element(By.XPATH, "/html/body/div[1]/div/div[2]/div[1]/div/div[2]/div[2]/div[4]/div/div/div[2]/div/div/div[3]/div[2]/div[1]/div[2]/div/div[1]/div/div/div/div[3]").text
        ids = re.findall(r'ENSG\d+', result)
        
        if len(ids) > 1:
            new_id = ids[1]
            print(f"Updated ID found: {new_id} for {ensembl_id}")
            return new_id
        else:
            print(f"No updated ID found for {ensembl_id}.")
            return None
    except Exception as e:
        print(f"Error fetching ID for {ensembl_id}: {e}")
        return None

found_last_id = False  # To track when we should start processing

for missing_id in missing_ids:
    if missing_id == last_processed_id:
        found_last_id = True
        continue  # Skip the last processed ID
    
    if not found_last_id:
        continue  # Skip IDs until we find the last processed one

    updated_id = fetch_updated_ensembl_id(missing_id)
    if updated_id:
        updated_ids.append({'original_id': missing_id, 'updated_id': updated_id})

driver.quit()

df = pd.DataFrame(updated_ids)
df.to_csv('updated_ids.csv', index=False)
